In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load the dataset
data = pd.DataFrame({
    'user_id': ['jonocole', 'jonocole', 'jonocole', 'jajo', 'jajo', 'isaac', 'isaac', 'lobsterclaw', 'lobsterclaw', 'massdosage'],
    'name': [
        'Panasonic 2 Ton AC', 'Hexzone Floor AC Stand', 'Hitachi Window AC',
        'Panasonic 2 Ton AC', 'Blue Star Split AC', 'Hitachi Window AC',
        'Hexzone Floor AC Stand', 'Panasonic 2 Ton AC', 'Blue Star Split AC',
        'Hitachi Window AC'
    ],
    'ratings': [4.1, 4.1, 2.7, 4.2, 4.1, 4.1, 4.0, 5.0, 3.9, 4.3]
})

# Pivot the data to create a user-item matrix
user_item_matrix = data.pivot(index='user_id', columns='name', values='ratings')

# Fill missing values with NaN
user_item_matrix = user_item_matrix.fillna(np.nan)

# Function to compute cosine similarity
def compute_cosine_similarity(matrix):
    sim_matrix = cosine_similarity(matrix.fillna(0))
    return pd.DataFrame(sim_matrix, index=matrix.index, columns=matrix.index)

# Function to compute Pearson correlation
def compute_pearson_similarity(matrix):
    return matrix.transpose().corr(method="pearson")

# Predict ratings
def predict_ratings(user_item_matrix, similarity_matrix, user_based=True):
    predictions = user_item_matrix.copy()
    for user in user_item_matrix.index:
        for item in user_item_matrix.columns:
            if pd.isna(user_item_matrix.loc[user, item]):
                if user_based:
                    sim_scores = similarity_matrix[user]
                    item_ratings = user_item_matrix[item]
                else:
                    sim_scores = similarity_matrix[item]
                    item_ratings = user_item_matrix.loc[user]

                valid_ratings = item_ratings[~item_ratings.isna()]
                valid_sim_scores = sim_scores[valid_ratings.index]

                if valid_sim_scores.sum() != 0:
                    predicted_value = (valid_sim_scores * valid_ratings).sum() / valid_sim_scores.sum()
                else:
                    predicted_value = np.nan

                predictions.loc[user, item] = predicted_value
    return predictions

# User-Based Collaborative Filtering
user_similarity_cosine = compute_cosine_similarity(user_item_matrix)
user_similarity_pearson = compute_pearson_similarity(user_item_matrix)

user_based_predictions_cosine = predict_ratings(user_item_matrix, user_similarity_cosine, user_based=True)
user_based_predictions_pearson = predict_ratings(user_item_matrix, user_similarity_pearson, user_based=True)

# Item-Based Collaborative Filtering
item_similarity_cosine = compute_cosine_similarity(user_item_matrix.transpose())
item_similarity_pearson = compute_pearson_similarity(user_item_matrix.transpose())

item_based_predictions_cosine = predict_ratings(user_item_matrix, item_similarity_cosine, user_based=False)
item_based_predictions_pearson = predict_ratings(user_item_matrix, item_similarity_pearson, user_based=False)

# Display results
print("User-Based CF Predictions (Cosine):")
print(user_based_predictions_cosine)
print("\nUser-Based CF Predictions (Pearson):")
print(user_based_predictions_pearson)

print("\nItem-Based CF Predictions (Cosine):")
print(item_based_predictions_cosine)
print("\nItem-Based CF Predictions (Pearson):")
print(item_based_predictions_pearson)


Initial Dataset:
                name main_category      sub_category  \
user_id                                               
user_1    Product 1   electronics       Televisions   
user_2    Product 2     furniture  Air Conditioners   
user_3    Product 3    appliances             Sofas   
user_4    Product 4     furniture  Air Conditioners   
user_5    Product 5     furniture     Refrigerators   
user_6    Product 6     furniture     Refrigerators   
user_7    Product 7     furniture  Air Conditioners   
user_8    Product 8   electronics  Air Conditioners   
user_9    Product 9    appliances       Televisions   
user_10  Product 10     furniture       Televisions   
user_11  Product 11     furniture     Refrigerators   
user_12  Product 12   electronics  Air Conditioners   
user_13  Product 13    appliances  Air Conditioners   
user_14  Product 14    appliances     Refrigerators   
user_15  Product 15    appliances             Sofas   
user_16  Product 16   electronics       Televis

ValueError: x and y must have length at least 2.